In [1]:
from google.colab import drive
drive.mount('/content/drive')
from tqdm import tqdm
import pandas as pd

Mounted at /content/drive


### Approach:

Divide the 11M word corpus into equally sized 10000 word bins. Count, within these bins, the number of 3, 4, and 5s. Assign to each bin the majority category.

Build a dict words `{word: bin}`

Build a dict bins `{bin: [l3, l4, l5]}`

Use the majority of each bin to label the words above.

In [2]:
cleaned_freqs = pd.read_csv('../data/freq/all_camelbert_freqs.csv')

In [3]:
cleaned_freqs['0']

0                     .
1                    في
2                     ،
3                    من
4                     "
               ...     
11724677    أبريسديانتو
11724678     أبروسكاتوا
11724679       إبداعتكن
11724680         أبجديو
11724681          أإشرف
Name: 0, Length: 11724682, dtype: object

In [4]:
import pickle

- Get all our known, labelled words and place them into the bins

In [9]:
with open('../data/levels_db/mle_max_aligned_model.pkl', 'rb') as f:
  known_levels = pickle.load(f)

In [ ]:
def get_levels_for_bins(n_bins):
  words = {}
  bins = {i: {3: 0, 4: 0, 5: 0} for i in range((len(cleaned_freqs['0'])+1) // n_bins)}
  not_in_corpus = 0

  for i in range(len(cleaned_freqs['0'])):
    words[cleaned_freqs['0'][i]] = i // n_bins

  for word, level in known_levels.items():
    try:
      bin = words[word]
      bins[bin][level] += 1
    except:
      not_in_corpus += 1
      pass
  return bins



In [19]:
def print_counts(counts):
  print(pd.DataFrame(counts.values()).value_counts())

- Do this for bins of equal size in terms of cumulative frequency

In [7]:
total_words = sum(cleaned_freqs['1'])

In [34]:
def level_full_pipeline_eq_sized(n_bins, oov_level = 5):

  bin_size = total_words/n_bins
  words = {}
  bins = {i: {3: 0, 4: 0, 5: 0} for i in range(0, n_bins)}

  bin_words = []

  current_sum = 0
  current_bin = 0
  words_in_bin = 0
  for word, freq in zip(cleaned_freqs['0'], cleaned_freqs['1']):
    words[word] = current_bin
    current_sum += freq
    words_in_bin += 1
    if current_sum > bin_size:
      current_sum = current_sum % bin_size
      current_bin += 1
      bin_words.append(words_in_bin)
      words_in_bin = 0

  for word, level in known_levels.items():
    try:
      bin = words[word]
      bins[bin][level] += 1
    except:
      pass


  no_hits_bins = 0

  bins_levelled = {}
  for bin in bins.keys():
    if list(bins[bin].values()).count(0) == 3:
      bins_levelled[bin] = oov_level
      no_hits_bins += 1
    else:
      bins_levelled[bin] = max(bins[bin].items(), key = lambda x: x[1])[0]

  levelled_words = {}
  for word in words:
    levelled_words[word] = bins_levelled[words[word]]

  print(no_hits_bins)
  return levelled_words, bin_words


In [35]:
bincounts = [10000, 5000, 2000, 1000, 500, 200, 100]
level_sets = {bc: level_full_pipeline_eq_sized(bc) for bc in bincounts}

4765
1934
603
255
105
32
10


In [36]:
with open('../data/freq/all_levels_freq_binning_cumulative.pkl', 'wb') as f:
  pickle.dump(level_sets, f)

In [37]:
for s in bincounts:
  print_counts(level_sets[s])

AttributeError: 'tuple' object has no attribute 'values'

In [44]:
cleaned_freqs['0'][1:100]

1          في
2           ،
3          من
4           "
5         على
       ...   
95        وهي
96    الحرمين
97        فيه
98        مثل
99    السعودي
Name: 0, Length: 99, dtype: object